In [10]:
import os
import json
import math
from PIL import Image  # Hanya menggunakan PIL untuk manipulasi gambar sederhana

# Folder yang berisi gambar dan file JSON yang berisi label
image_train_folder = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/train'
image_test_folder = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/valid'
json_train_file = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/train/annotations_color_tain.json'
json_test_file = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/valid/annotations_color_valid.json'


In [3]:
# Fungsi untuk memuat data gambar dan label dari file JSON
def load_data_from_json_and_images(image_folder, json_file):
    images = []
    labels = []
    filenames = []  # Untuk menyimpan nama file gambar
     
    # Membaca file JSON yang berisi anotasi
    with open(json_file) as f:
        data = json.load(f)

    # Membaca gambar dan label dari JSON
    for annotation in data['annotations']:
        image_filename = data['images'][annotation['image_id']]['file_name']  # Ambil nama file gambar berdasarkan image_id
        label = annotation['color_label']  # Label yang berhubungan dengan gambar
        
        # Memuat gambar dari folder
        image_path = os.path.join(image_folder, image_filename)
        image = Image.open(image_path)  # Menggunakan PIL untuk membuka gambar
        
        # Ubah ukuran gambar jika perlu (agar ukuran konsisten)
        image_resized = image.resize((128, 128))  # Mengubah ukuran gambar menjadi 64x64
        
        # Flatten gambar menjadi array 1D
        image_flattened = list(image_resized.getdata())  # Mengambil pixel data dalam urutan (R, G, B)
        image_flattened = [pixel[0] / 255 + pixel[1] / 255 + pixel[2] / 255 for pixel in image_flattened]


        images.append(image_flattened)
        labels.append(label)
    
    return images, labels

In [12]:
# Fungsi Euclidean Distance
def euclidean_distance(vec1, vec2):
    # Menghitung jarak Euclidean antara dua vektor
    return math.sqrt(sum((a - b) ** 2 for a, b in zip(vec1, vec2)))


In [13]:
# Fungsi KNN Manual
def knn_classify(test_data, train_data, train_labels, k=3):
    distances = []
    
    # Menghitung jarak Euclidean antara data uji dan setiap data latih
    for i in range(len(train_data)):
        dist = euclidean_distance(test_data, train_data[i])
        distances.append((dist, train_labels[i]))
    
    # Urutkan berdasarkan jarak dan ambil k terdekat
    distances.sort(key=lambda x: x[0])
    k_nearest = distances[:k]
    
    # Voting label terbanyak
    label_votes = {}
    for _, label in k_nearest:
        if label in label_votes:
            label_votes[label] += 1
        else:
            label_votes[label] = 1
    
    return max(label_votes, key=label_votes.get)

In [14]:
# Muat data latih
train_data, train_labels = load_data_from_json_and_images(image_train_folder, json_train_file)

# Muat data uji
test_data, test_labels = load_data_from_json_and_images(image_test_folder, json_test_file)


In [15]:
# Evaluasi KNN
k = 5  # Tetangga terdekat
correct_predictions = 0

for i, test_sample in enumerate(test_data):
    predicted_label = knn_classify(test_sample, train_data, train_labels, k)
    print(f"Test Image {i+1}: Predicted Label = {predicted_label}, True Label = {test_labels[i]}")
    
    # Hitung jumlah prediksi benar
    if predicted_label == test_labels[i]:
        correct_predictions += 1

# Akurasi
accuracy = correct_predictions / len(test_labels) * 100
print(f"Accuracy: {accuracy:.2f}%")

Test Image 1: Predicted Label = Blue, True Label = Red
Test Image 2: Predicted Label = Red, True Label = Red
Test Image 3: Predicted Label = Red, True Label = Red
Test Image 4: Predicted Label = Red, True Label = Green
Test Image 5: Predicted Label = Red, True Label = Blue
Test Image 6: Predicted Label = Red, True Label = Green
Test Image 7: Predicted Label = Red, True Label = Red
Test Image 8: Predicted Label = Red, True Label = Red
Test Image 9: Predicted Label = Blue, True Label = White
Test Image 10: Predicted Label = Green, True Label = Green
Test Image 11: Predicted Label = Red, True Label = Blue
Test Image 12: Predicted Label = Blue, True Label = Green
Test Image 13: Predicted Label = Red, True Label = Green
Test Image 14: Predicted Label = Red, True Label = Red
Test Image 15: Predicted Label = Red, True Label = Blue
Test Image 16: Predicted Label = Red, True Label = Green
Test Image 17: Predicted Label = Black, True Label = Blue
Test Image 18: Predicted Label = Red, True Label 

In [5]:
import os
import json
from skimage import io
from skimage.transform import resize

# Folder yang berisi gambar dan file JSON yang berisi label
image_train_folder = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/train'
image_test_folder = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/valid' 
json_train_file = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/train/annotations_color_valid_accurate.json'  
json_test_file = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/valid/annotations_color_valid_accurate.json'  

# Fungsi untuk memuat data gambar dan label dari file JSON
def load_data_from_json_and_images(image_folder, json_file):
    images = []
    labels = []
    filenames = []  # Untuk menyimpan nama file gambar
    
    # Membaca file JSON yang berisi anotasi
    with open(json_file) as f:
        data = json.load(f)

    # Membaca gambar dan label dari JSON
    for annotation in data['annotations']:
        image_filename = data['images'][annotation['image_id']]['file_name']  # Ambil nama file gambar berdasarkan image_id
        label = annotation['color_label']  # Label yang berhubungan dengan gambar
        
        # Memuat gambar dari folder
        image_path = os.path.join(image_folder, image_filename)
        image = io.imread(image_path)
        
        # Ubah ukuran gambar jika perlu (agar ukuran konsisten)
        image_resized = resize(image, (64, 64))  # Mengubah ukuran gambar menjadi 64x64
        
        # Flatten gambar menjadi array 1D
        image_flattened = image_resized.flatten()
        
        images.append(image_flattened)
        labels.append(label)
        filenames.append(image_filename)
    
    return images, labels,filenames

# Fungsi untuk menghitung jarak Euclidean antara dua vektor
def euclidean_distance(vec1, vec2):
    return sum((a - b) ** 2 for a, b in zip(vec1, vec2)) ** 0.5

# Fungsi KNN
def knn_classify(test_data, train_data, train_labels, k=3):
    distances = []
    
    # Hitung jarak antara data uji dan semua data latih
    for i in range(len(train_data)):
        dist = euclidean_distance(test_data, train_data[i])
        distances.append((dist, train_labels[i]))
    
    # Urutkan jarak dan ambil k terdekat
    distances.sort(key=lambda x: x[0])
    k_nearest = distances[:k]
    
    # Voting label terbanyak
    label_votes = {}
    for _, label in k_nearest:
        if label in label_votes:
            label_votes[label] += 1
        else:
            label_votes[label] = 1
    
    # Kembalikan label dengan jumlah suara terbanyak
    return max(label_votes, key=label_votes.get)

# Muat data latih
train_data, train_labels = load_data_from_json_and_images(image_train_folder, json_train_file)

# Muat data uji
test_data, test_labels = load_data_from_json_and_images(image_test_folder, json_test_file)

# Evaluasi KNN
k = 5  # Tetangga terdekat
correct_predictions = 0

for i, test_sample in enumerate(test_data):
    predicted_label = knn_classify(test_sample, train_data, train_labels, k)
    
    # Cetak nama file gambar, label prediksi, dan label yang benar
    print(f"Test Image {test_filenames[i]}: Predicted Label = {predicted_label}, True Label = {test_labels[i]}")
    
    # Hitung jumlah prediksi benar
    if predicted_label == test_labels[i]:
        correct_predictions += 1

# Akurasi
accuracy = correct_predictions / len(test_labels) * 100
print(f"Accuracy: {accuracy:.2f}%")


ValueError: too many values to unpack (expected 2)

In [6]:
import os
import json
import numpy as np
from skimage import io
from skimage.transform import resize

# Folder yang berisi gambar dan file JSON yang berisi label
image_train_folder = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/train'
image_test_folder = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/valid' 
json_train_file = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/train/annotations_color_valid_accurate.json'  
json_test_file = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/data/containernumbers_container-serials/valid/annotations_color_valid_accurate.json'  

# Fungsi untuk memuat data gambar dan label dari file JSON
def load_data_from_json_and_images(image_folder, json_file):
    images = []
    labels = []
    filenames = []  # Untuk menyimpan nama file gambar
    
    # Membaca file JSON yang berisi anotasi
    with open(json_file) as f:
        data = json.load(f)

    # Membaca gambar dan label dari JSON
    for annotation in data['annotations']:
        image_filename = data['images'][annotation['image_id']]['file_name']  # Ambil nama file gambar berdasarkan image_id
        label = annotation['color_label']  # Label yang berhubungan dengan gambar
        
        # Memuat gambar dari folder
        image_path = os.path.join(image_folder, image_filename)
        image = io.imread(image_path)
        
        # Ubah ukuran gambar jika perlu (agar ukuran konsisten)
        image_resized = resize(image, (64, 64))  # Mengubah ukuran gambar menjadi 64x64
        
        # Flatten gambar menjadi array 1D
        image_flattened = image_resized.flatten()
        
        images.append(image_flattened)
        labels.append(label)
        filenames.append(image_filename)  # Simpan nama file gambar
    
    return np.array(images), labels, filenames  # Kembalikan filenames juga

# Fungsi Euclidean Distance
def euclidean_distance(vec1, vec2):
    return np.sqrt(np.sum((vec1 - vec2) ** 2))

# Fungsi KNN Manual
def knn_classify(test_data, train_data, train_labels, k=3):
    distances = []
    for i in range(len(train_data)):
        dist = euclidean_distance(test_data, train_data[i])
        distances.append((dist, train_labels[i]))
    
    # Urutkan berdasarkan jarak dan ambil k terdekat
    distances.sort(key=lambda x: x[0])
    k_nearest = distances[:k]
    
    # Voting label terbanyak
    label_votes = {}
    for _, label in k_nearest:
        if label in label_votes:
            label_votes[label] += 1
        else:
            label_votes[label] = 1
    
    return max(label_votes, key=label_votes.get)

# Muat data latih
train_data, train_labels, train_filenames = load_data_from_json_and_images(image_train_folder, json_train_file)

# Muat data uji
test_data, test_labels, test_filenames = load_data_from_json_and_images(image_test_folder, json_test_file)

# Evaluasi KNN
k = 5  # Tetangga terdekat
correct_predictions = 0

for i, test_sample in enumerate(test_data):
    predicted_label = knn_classify(test_sample, train_data, train_labels, k)
    
    # Cetak nama file gambar, label prediksi, dan label yang benar
    print(f"Test Image {test_filenames[i]}: Predicted Label = {predicted_label}, True Label = {test_labels[i]}")
    
    # Hitung jumlah prediksi benar
    if predicted_label == test_labels[i]:
        correct_predictions += 1

# Akurasi
accuracy = correct_predictions / len(test_labels) * 100
print(f"Accuracy: {accuracy:.2f}%")


Test Image 1-123204001-OCR-LF-C01_jpg.rf.3ebf94d4d30d7766258de443d5b1dc32.jpg: Predicted Label = Other, True Label = Red
Test Image 1-123303001-OCR-LB-C02_jpg.rf.e20957316647a9514bc278222c0724a9.jpg: Predicted Label = Red, True Label = Red
Test Image 1-123337001-OCR-AH-A01_jpg.rf.9b3ee4fbba917ec73542a59e6c56097b.jpg: Predicted Label = Other, True Label = Other
Test Image 1-123402001-OCR-LB-C02_jpg.rf.f381c410f3cf6e1e03a1d9887902bbad.jpg: Predicted Label = Red, True Label = Blue
Test Image 1-123402001-OCR-RF-D01_jpg.rf.bb107fbd3c8e40bef2101f8e0f6cb9fa.jpg: Predicted Label = Blue, True Label = Blue
Test Image 1-123407001-OCR-AS-B01_jpg.rf.a530131fafebf7e7ef20c0479bae3fa2.jpg: Predicted Label = Black, True Label = Other
Test Image 1-123407001-OCR-LF-C01_jpg.rf.f5582352ba9b9f0972b1ca42a39cec28.jpg: Predicted Label = Other, True Label = Other
Test Image 1-123504001-OCR-LB-C02_jpg.rf.25a9baf656457e9fda992b3e2d90e70d.jpg: Predicted Label = Red, True Label = Red
Test Image 1-123608001-OCR-LB-C

In [4]:
import os
import json
import numpy as np
from skimage import io
from skimage.transform import resize

# Folder yang berisi gambar dan file JSON yang berisi label
image_folder = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/Container-Detector-Use-KNN/data'
json_file = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/Container-Detector-Use-KNN/data/annotations_color.json'
# Fungsi untuk memuat data gambar dan label dari file JSON
def load_data_from_json_and_images(image_folder, json_file):
    images = []
    labels = []
    filenames = []  # Untuk menyimpan nama file gambar
    
    # Membaca file JSON yang berisi anotasi
    with open(json_file) as f:
        data = json.load(f)

    # Membaca gambar dan label dari JSON
    for annotation in data['annotations']:
        image_filename = data['images'][annotation['image_id']]['file_name']  # Ambil nama file gambar berdasarkan image_id
        label = annotation['color_label']  # Label yang berhubungan dengan gambar
        
        # Memuat gambar dari folder
        image_path = os.path.join(image_folder, image_filename)
        image = io.imread(image_path)
        
        # Ubah ukuran gambar jika perlu (agar ukuran konsisten)
        image_resized = resize(image, (64, 64))  # Mengubah ukuran gambar menjadi 64x64
        
        # Flatten gambar menjadi array 1D
        image_flattened = image_resized.flatten()
        
        images.append(image_flattened)
        labels.append(label)
        filenames.append(image_filename)  # Simpan nama file gambar
    
    return np.array(images), labels, filenames  # Kembalikan filenames juga

# Fungsi Euclidean Distance
def euclidean_distance(vec1, vec2):
    return np.sqrt(np.sum((vec1 - vec2) ** 2))

# Fungsi KNN Manual
def knn_classify(test_data, train_data, train_labels, k=3):
    distances = []
    for i in range(len(train_data)):
        dist = euclidean_distance(test_data, train_data[i])
        distances.append((dist, train_labels[i]))
    
    # Urutkan berdasarkan jarak dan ambil k terdekat
    distances.sort(key=lambda x: x[0])
    k_nearest = distances[:k]
    
    # Voting label terbanyak
    label_votes = {}
    for _, label in k_nearest:
        if label in label_votes:
            label_votes[label] += 1
        else:
            label_votes[label] = 1
    
    return max(label_votes, key=label_votes.get)

# Muat semua data (gambar dan label)
images, labels, filenames = load_data_from_json_and_images(image_folder, json_file)

# Membagi data menjadi data latih (90%) dan data uji (10%) secara manual
data_size = len(images)
indices = np.random.permutation(data_size)  # Mengacak indeks
train_size = int(data_size * 0.9)  # 90% untuk data latih
train_indices = indices[:train_size]
test_indices = indices[train_size:]

train_data = images[train_indices]
train_labels = [labels[i] for i in train_indices]
train_filenames = [filenames[i] for i in train_indices]

test_data = images[test_indices]
test_labels = [labels[i] for i in test_indices]
test_filenames = [filenames[i] for i in test_indices]

# Evaluasi KNN
k = 5  # Tetangga terdekat
correct_predictions = 0

for i, test_sample in enumerate(test_data):
    predicted_label = knn_classify(test_sample, train_data, train_labels, k)
    
    # Cetak nama file gambar, label prediksi, dan label yang benar
    print(f"Test Image {test_filenames[i]}: Predicted Label = {predicted_label}, True Label = {test_labels[i]}")
    
    # Hitung jumlah prediksi benar
    if predicted_label == test_labels[i]:
        correct_predictions += 1

# Akurasi
accuracy = correct_predictions / len(test_labels) * 100
print(f"Accuracy: {accuracy:.2f}%")


Test Image Container-439_jpg.rf.410dcfbf1229d704728b9836b583947d.jpg: Predicted Label = White, True Label = Green
Test Image Container-699_jpg.rf.fab76afa2c58756ef679955cb6b9eecd.jpg: Predicted Label = Red, True Label = Red
Test Image DFSU7170569-3-_jpg.rf.fcfc0d2cbebeea4bc6a98ee4dee893a2.jpg: Predicted Label = Black, True Label = Black
Test Image BSIU2952609-4-_jpg.rf.e3ca54ff4d3f8cde9e9b81b7359f7453.jpg: Predicted Label = Black, True Label = Black
Test Image BSIU3182950-3-_638x480_jpg.rf.0e399fec3e5b404f8c64ff03c5e0b866.jpg: Predicted Label = Black, True Label = Black
Test Image 1-123147001-OCR-LF-C01_jpg.rf.d955762af70c53bd23c179b1262d4a04.jpg: Predicted Label = Other, True Label = Red
Test Image IMG_20221207_124709_638x480_jpg.rf.c6d5191ed1b4112d45f89570e2aea399.jpg: Predicted Label = White, True Label = White
Test Image CCLU7932702-4-_jpg.rf.48d9202c926dd45230a444cfd55e2c0a.jpg: Predicted Label = White, True Label = White
Test Image 20191029-180340-01-3-R_jpg.rf.66e996d494c7b85fcd

In [27]:
import os
import json
from skimage import io
from skimage.transform import resize

# Folder yang berisi gambar dan file JSON yang berisi label
image_folder = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/Container-Detector-Use-KNN/data'
json_file = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/Container-Detector-Use-KNN/titik_tengah.json'

# Fungsi untuk memuat data gambar dan label dari file JSON
def load_data_from_json_and_images(image_folder, json_file):
    images = []
    labels = []
    filenames = []  # Untuk menyimpan nama file gambar
    
    # Membaca file JSON yang berisi anotasi
    with open(json_file) as f:
        data = json.load(f)

    # Membaca gambar dan label dari JSON
    for annotation in data['annotations']:
        image_filename = data['images'][annotation['image_id']]['file_name']  # Ambil nama file gambar berdasarkan image_id
        label = annotation['color_label']  # Label yang berhubungan dengan gambar
        
        # Memuat gambar dari folder
        image_path = os.path.join(image_folder, image_filename)
        image = io.imread(image_path)
        
        # Ubah ukuran gambar jika perlu (agar ukuran konsisten)
        image_resized = resize(image, (64, 64))  # Mengubah ukuran gambar menjadi 64x64
        
        # Flatten gambar menjadi array 1D dengan list
        image_flattened = image_resized.flatten().tolist()  # Konversi hasil flatten ke list
        
        images.append(image_flattened)
        labels.append(label)
        filenames.append(image_filename)  # Simpan nama file gambar
    
    return images, labels, filenames  # Kembalikan filenames juga

# Fungsi Euclidean Distance tanpa 'math'
def euclidean_distance(vec1, vec2):
    # Hitung jarak Euclidean antara dua list
    sum_squared = sum((a - b) ** 2 for a, b in zip(vec1, vec2))
    return sum_squared ** 0.5  # Menggunakan perpangkatan dengan 0.5 untuk akar kuadrat

# Fungsi KNN Manual
def knn_classify(test_data, train_data, train_labels, k=3):
    distances = []
    for i in range(len(train_data)):
        dist = euclidean_distance(test_data, train_data[i])
        distances.append((dist, train_labels[i]))
    
    # Urutkan berdasarkan jarak dan ambil k terdekat
    distances.sort(key=lambda x: x[0])
    k_nearest = distances[:k]
    
    # Voting label terbanyak
    label_votes = {}
    for _, label in k_nearest:
        if label in label_votes:
            label_votes[label] += 1
        else:
            label_votes[label] = 1
    
    return max(label_votes, key=label_votes.get)

# Muat semua data (gambar dan label)
images, labels, filenames = load_data_from_json_and_images(image_folder, json_file)

# Membagi data menjadi data latih (90%) dan data uji (10%) secara manual
data_size = len(images)
indices = list(range(data_size))
# Mengacak indeks
import random
random.shuffle(indices)

train_size = int(data_size * 0.9)  # 90% untuk data latih
train_indices = indices[:train_size]
test_indices = indices[train_size:]

train_data = [images[i] for i in train_indices]
train_labels = [labels[i] for i in train_indices]
train_filenames = [filenames[i] for i in train_indices]

test_data = [images[i] for i in test_indices]
test_labels = [labels[i] for i in test_indices]
test_filenames = [filenames[i] for i in test_indices]

# Evaluasi KNN
k = 5  # Tetangga terdekat
correct_predictions = 0

for i, test_sample in enumerate(test_data):
    predicted_label = knn_classify(test_sample, train_data, train_labels, k)
    
    # Cetak nama file gambar, label prediksi, dan label yang benar
    print(f"Test Image {test_filenames[i]}: Predicted Label = {predicted_label}, True Label = {test_labels[i]}")
    
    # Hitung jumlah prediksi benar
    if predicted_label == test_labels[i]:
        correct_predictions += 1

# Akurasi
accuracy = correct_predictions / len(test_labels) * 100
print(f"Accuracy: {accuracy:.2f}%")


Test Image OOCU8973210-4-_jpg.rf.49598f533e5833da443bcaed30338ffb.jpg: Predicted Label = Red, True Label = White
Test Image 20191029-195857-01-3-R_jpg.rf.6e01ff5a149bc20846565318a6aeb519.jpg: Predicted Label = Red, True Label = Red
Test Image 1-122923001-OCR-AH-A01_jpg.rf.a250e0ee64a2047e8a3534bae814e153.jpg: Predicted Label = Other, True Label = Other
Test Image BMOU2454010-1-_jpg.rf.c4103ab45069464c43a6200f6a6125f9.jpg: Predicted Label = Black, True Label = Black
Test Image CAXU-723870-9-0-1-0-1_bmp.rf.44ecb1a3fe1d4c12c42b1bfd9037f221.jpg: Predicted Label = White, True Label = Other
Test Image 20191029-221243-01-4-R_jpg.rf.8fb440c0f2f4d094df54f412050183c2.jpg: Predicted Label = White, True Label = White
Test Image AXIU4224514-1-_jpg.rf.e547edc6b0fb1575bd1df834b5c68ac3.jpg: Predicted Label = Black, True Label = Red
Test Image Container-520_jpg.rf.dda1658d550e46f9a66ab5f7e31be60e.jpg: Predicted Label = Blue, True Label = Blue
Test Image CSNU8510989-1-_jpg.rf.cc500ece12fcbc83f569e8eb297

In [18]:
import os
import json
import random
import math

# Fungsi untuk memuat data gambar dan label dari file JSON
def load_data_from_json_and_images(image_folder, json_file):
    images = []
    labels = []
    filenames = []  # Untuk menyimpan nama file gambar
    
    # Membaca file JSON yang berisi anotasi
    with open(json_file) as f:
        data = json.load(f)

    # Membaca gambar dan label dari JSON
    for annotation in data['annotations']:
        image_filename = data['images'][annotation['image_id']]['file_name']  # Ambil nama file gambar berdasarkan image_id
        label = annotation['color_label']  # Label yang berhubungan dengan gambar
        
        # Memuat gambar dari folder (misalnya gambar dalam bentuk list atau data pixel)
        image_path = os.path.join(image_folder, image_filename)
        with open(image_path, 'rb') as img_file:
            # Membaca gambar sebagai byte (akan perlu diproses lebih lanjut untuk mengekstrak nilai pixel secara manual)
            image_data = img_file.read()
        
        # Untuk contoh, kita anggap gambar sudah dalam bentuk array 1D (flatten) sebagai fitur
        # Misalnya gambar diratakan dan diubah menjadi representasi list pixel
        # Anda perlu mengekstrak fitur gambar dengan cara lain jika ingin memproses gambar sungguhan tanpa pustaka
        image_flattened = list(image_data[:512])  # Placeholder: ambil 64 byte pertama sebagai contoh data
        
        images.append(image_flattened)
        labels.append(label)
        filenames.append(image_filename)  # Simpan nama file gambar
    
    return images, labels, filenames  # Kembalikan filenames juga

# Fungsi Naive Bayes tanpa library eksternal
class NaiveBayes:
    def __init__(self):
        self.class_probs = {}
        self.feature_probs = {}

    def fit(self, X, y):
        # Menghitung prior probabilities (P(C_k)) untuk setiap kelas
        class_counts = {}
        feature_counts = {}
        num_samples = len(X)

        for i in range(num_samples):
            label = y[i]
            if label not in class_counts:
                class_counts[label] = 0
                feature_counts[label] = []
            class_counts[label] += 1
            feature_counts[label].append(X[i])

        # Prior probabilities P(C_k)
        self.class_probs = {k: v / num_samples for k, v in class_counts.items()}

        # Likelihood P(X|C_k), kita anggap fitur independen
        self.feature_probs = {}

        for label, features in feature_counts.items():
            # Menyimpan probabilitas untuk setiap fitur pada kelas tertentu
            feature_probs_label = []
            features_array = features  # Fitur per kelas
            feature_means = self.calculate_mean(features_array)
            feature_stds = self.calculate_std(features_array, feature_means)
            feature_probs_label = (feature_means, feature_stds)
            self.feature_probs[label] = feature_probs_label

    def predict(self, X):
        predictions = []
        for sample in X:
            # Menghitung probabilitas untuk setiap kelas dan memilih kelas dengan probabilitas tertinggi
            class_scores = {}
            for label in self.class_probs:
                # Mulai dengan prior P(C_k)
                score = math.log(self.class_probs[label])
                
                # Menghitung likelihood P(X|C_k) untuk setiap fitur (menggunakan distribusi normal)
                feature_means, feature_stds = self.feature_probs[label]
                for i in range(len(sample)):
                    prob = self.gaussian_pdf(sample[i], feature_means[i], feature_stds[i])
                    score += math.log(prob)

                class_scores[label] = score

            # Kelas dengan skor tertinggi
            predicted_class = max(class_scores, key=class_scores.get)
            predictions.append(predicted_class)

        return predictions

    def gaussian_pdf(self, x, mean, std):
    # Menghindari pembagian dengan nol jika std == 0
        if std == 0:
            return 1  # Jika standar deviasi nol, anggap probabilitasnya 1 (atau bisa diganti dengan nilai yang sesuai)
        
        exponent = math.exp(- ((x - mean) ** 2) / (2 * std ** 2))
        return (1 / (math.sqrt(2 * math.pi) * std)) * exponent


    def calculate_mean(self, features):
        # Menghitung rata-rata setiap fitur
        features_array = list(zip(*features))  # Transpose untuk mendapatkan fitur per kolom
        means = [sum(feature) / len(feature) for feature in features_array]
        return means

    def calculate_std(self, features, means):
    # Menghitung standar deviasi setiap fitur
        features_array = list(zip(*features))  # Transpose untuk mendapatkan fitur per kolom
        stds = []
        for feature, mean in zip(features_array, means):
            variance = sum((x - mean) ** 2 for x in feature) / len(feature)
            std = math.sqrt(variance)
            
            # Jika standar deviasi adalah nol, ubah menjadi nilai kecil untuk menghindari pembagian dengan nol
            if std == 0:
                std = 1e-6  # Nilai kecil untuk menghindari pembagian dengan nol
            
            stds.append(std)
        
        return stds


# Fungsi untuk membagi data menjadi data latih dan data uji
def train_test_split(X, y, test_size=0.1):
    data_size = len(X)
    indices = list(range(data_size))
    random.shuffle(indices)
    
    train_size = int(data_size * (1 - test_size))
    train_indices = indices[:train_size]
    test_indices = indices[train_size:]
    
    train_data = [X[i] for i in train_indices]
    train_labels = [y[i] for i in train_indices]
    
    test_data = [X[i] for i in test_indices]
    test_labels = [y[i] for i in test_indices]
    
    return train_data, test_data, train_labels, test_labels

# Muat semua data (gambar dan label)
image_folder = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/Container-Detector-Use-KNN/data'
json_file = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/Container-Detector-Use-KNN/data/annotations_color.json'
images, labels, filenames = load_data_from_json_and_images(image_folder, json_file)

# Membagi data menjadi data latih dan data uji
train_data, test_data, train_labels, test_labels = train_test_split(images, labels, test_size=0.1)

# Latih model Naive Bayes
nb = NaiveBayes()
nb.fit(train_data, train_labels)

# Prediksi dengan data uji
predictions = nb.predict(test_data)

# Evaluasi: hitung akurasi
correct_predictions = sum([1 if predictions[i] == test_labels[i] else 0 for i in range(len(test_labels))])
accuracy = correct_predictions / len(test_labels) * 100
print(f"Akurasi Naive Bayes: {accuracy:.2f}%")


Akurasi Naive Bayes: 27.80%


In [19]:
import os
import json
import random
import numpy as np
import math
from collections import defaultdict

# Fungsi untuk memuat data gambar dan label dari file JSON
def load_data_from_json_and_images(image_folder, json_file):
    images = []
    labels = []
    filenames = []
    
    with open(json_file) as f:
        data = json.load(f)

    # Memuat gambar dan label
    for annotation in data['annotations']:
        image_filename = data['images'][annotation['image_id']]['file_name']
        label = annotation['color_label']
        
        # Path gambar
        image_path = os.path.join(image_folder, image_filename)
        image_data = load_image(image_path)
        
        # Menambahkan data ke list
        images.append(image_data)
        labels.append(label)
        filenames.append(image_filename)
    
    return images, labels, filenames

# Fungsi untuk memuat gambar dan mengubahnya menjadi fitur warna
def load_image(image_path):
    # Membaca gambar, mengonversi ke array RGB
    image = np.array([[np.random.randint(0, 256) for _ in range(3)] for _ in range(64)])  # Dummy image data (64x3)
    return image.flatten()  # Meratakan gambar menjadi vektor 1D sebagai fitur

# Fungsi untuk membagi data menjadi data latih dan data uji
def train_test_split(X, y, test_size=0.1):
    data_size = len(X)
    indices = list(range(data_size))
    random.shuffle(indices)
    
    train_size = int(data_size * (1 - test_size))
    train_indices = indices[:train_size]
    test_indices = indices[train_size:]
    
    train_data = [X[i] for i in train_indices]
    train_labels = [y[i] for i in train_indices]
    
    test_data = [X[i] for i in test_indices]
    test_labels = [y[i] for i in test_indices]
    
    return train_data, test_data, train_labels, test_labels


In [20]:
class NaiveBayes:
    def __init__(self):
        self.class_probs = {}
        self.feature_probs = {}

    def fit(self, X, y):
        class_counts = defaultdict(int)
        feature_counts = defaultdict(list)
        num_samples = len(X)

        # Menghitung frekuensi setiap kelas dan fitur
        for i in range(num_samples):
            label = y[i]
            class_counts[label] += 1
            feature_counts[label].append(X[i])

        # Menghitung prior probabilities P(C_k)
        self.class_probs = {k: v / num_samples for k, v in class_counts.items()}

        # Menghitung probabilitas untuk setiap fitur
        self.feature_probs = {}
        for label, features in feature_counts.items():
            feature_means = self.calculate_mean(features)
            feature_stds = self.calculate_std(features, feature_means)
            self.feature_probs[label] = (feature_means, feature_stds)

    def predict(self, X):
        predictions = []
        for sample in X:
            class_scores = {}
            for label in self.class_probs:
                score = math.log(self.class_probs[label])
                feature_means, feature_stds = self.feature_probs[label]
                for i in range(len(sample)):
                    prob = self.gaussian_pdf(sample[i], feature_means[i], feature_stds[i])
                    score += math.log(prob)
                class_scores[label] = score

            predicted_class = max(class_scores, key=class_scores.get)
            predictions.append(predicted_class)

        return predictions

    def gaussian_pdf(self, x, mean, std):
        if std == 0:
            return 1  # Menangani pembagian dengan nol jika std == 0
        exponent = math.exp(- ((x - mean) ** 2) / (2 * std ** 2))
        return (1 / (math.sqrt(2 * math.pi) * std)) * exponent

    def calculate_mean(self, features):
        features_array = list(zip(*features))  # Transpose untuk mendapatkan fitur per kolom
        means = [sum(feature) / len(feature) for feature in features_array]
        return means

    def calculate_std(self, features, means):
        features_array = list(zip(*features))  # Transpose untuk mendapatkan fitur per kolom
        stds = [math.sqrt(sum((x - mean) ** 2 for x in feature) / len(feature)) for feature, mean in zip(features_array, means)]
        return stds


In [24]:
# Muat semua data (gambar dan label)
image_folder = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/Container-Detector-Use-KNN/data'
json_file = 'D:/Serba Serbi Kuliah/MATERI KULIAH SMT 5/AI/UAS_AI/Container-Detector-Use-KNN/data/annotations_color.json'
images, labels, filenames = load_data_from_json_and_images(image_folder, json_file)

# Membagi data menjadi data latih dan data uji
train_data, test_data, train_labels, test_labels = train_test_split(images, labels, test_size=0.1)

# Latih model Naive Bayes
nb = NaiveBayes()
nb.fit(train_data, train_labels)

# Prediksi dengan data uji
predictions = nb.predict(test_data)

# Evaluasi: hitung akurasi
correct_predictions = sum([1 if predictions[i] == test_labels[i] else 0 for i in range(len(test_labels))])
accuracy = correct_predictions / len(test_labels) * 100
print(f"Akurasi Naive Bayes: {accuracy:.2f}%")


Akurasi Naive Bayes: 27.48%
